<span style="color:#7200CC; font-size:30px;">House Pricing Prediction - Full Data Science Pipeline</span>

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)

<span style="color:green; font-size:25px;">Data Loading</span>

In [2]:
df1 = pd.read_csv("house_prices_data.csv")
df1.head()

FileNotFoundError: [Errno 2] No such file or directory: 'house_prices_data.csv'

In [ ]:
df1.shape

In [ ]:
df1.describe()

<span style="color:green; font-size:25px;">Data Cleaning</span>

In [ ]:
df1['area_type'].value_counts()

In [ ]:
df2 = df1.drop(['area_type','society','balcony','availability'],axis='columns')
df2.shape

In [ ]:
df2.isnull().sum()

In [ ]:
df3 = df2.dropna()
df3.isnull().sum()

In [ ]:
df3.shape

In [ ]:
df3['size'].unique()

In [ ]:
# df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))
df3.loc[:, 'bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))


In [ ]:
df3.head()

In [ ]:
df3.bhk.unique()

In [ ]:
df3[df3['bhk'] > 20]

In [ ]:
df3.total_sqft.unique()

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
df3[~df3['total_sqft'].apply(is_float)].head(3)

In [ ]:
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        y = (float(tokens[0]) + float(tokens[1])) / 2
        return y
    try:
        return float(x)
    except:
        return None
        

In [ ]:
df4 = df3.copy()
df4['total_sqft'] = df4['total_sqft'].apply(convert_sqft_to_num)
df4.head(3)

In [ ]:
df4.loc[137]


<span style="color:green; font-size:25px;">Feature Engineering: Add new feature called price per square feet</span>

In [ ]:
df5 = df4.copy()
df5['price_per_sqft'] = df5['price'] * 100000 / df5['total_sqft']
df5.head(3)

In [ ]:
len(df5.location.unique())

In [ ]:
df5.location = df5.location.apply(lambda x: x.strip())
len(df5.location.unique())

<span style="color:green; font-size:25px;">Feature Engineering: Dimensionality Reduction</span>

In [ ]:
location_stats = df5.location.value_counts()
location_stats

In [ ]:
len(location_stats[location_stats < 15])

In [ ]:
location_count_less_than_15 = location_stats[location_stats <= 15]

In [ ]:
df5.location = df5.location.apply(lambda x: 'other' if x in location_count_less_than_15 else x)

In [ ]:
df5.head()

In [ ]:
len(df5.location.unique())

<span style="color:green; font-size:25px;">Outlier Removal: Using Business Logic</span>

#### "The area of an apartment / the number of bedroom < 300 sqft" is unusual. Data that does not fit this criteria need to be removed.

In [ ]:
df5[df5.total_sqft/df5.bhk < 300].head()

In [ ]:
df5.shape

In [ ]:
df6 = df5[~(df5.total_sqft/df5.bhk < 300)]

In [ ]:
df6.shape

<span style="color:green; font-size:25px;">Outlier Removal: Using Standard Deviation and Mean</span>

In [ ]:
df6.price_per_sqft.describe()

#### Here I found that min price per sqft is 267 currency/sqft whereas max is 176470 currency/sqft, this shows a wide variation in property prices. Therefore, I removed outliers per location using mean and one standard deviation.

In [ ]:
def remove_price_per_sqft_outliers(df):
    new_df = pd.DataFrame()
    for key, broken_df in df.groupby('location'):
        mean = np.mean(broken_df.price_per_sqft)
        std_dev = np.std(broken_df.price_per_sqft)
        reduced_df = broken_df[ ((broken_df.price_per_sqft) < (mean + std_dev) ) & ( (broken_df.price_per_sqft) > (mean - std_dev)) ]
        new_df = pd.concat([new_df, reduced_df], ignore_index = True)
    return new_df

df7 = remove_price_per_sqft_outliers(df6)
df7.shape

In [ ]:
df7.describe()

#### Let's check for a given location how the 2 BHK and 3 BHK property prices look like

In [ ]:
def plot_scatter_chart(df,location):
    bhk_2 = df[(df.location==location) & (df.bhk==2)]
    bhk_3 = df[(df.location==location) & (df.bhk==3)]
    matplotlib.rcParams['figure.figsize'] = (12,6)
    plt.scatter(bhk_2.total_sqft,bhk_2.price,color='blue',label='2 BHK', s=50)
    plt.scatter(bhk_3.total_sqft,bhk_3.price,marker='+', color='green',label='3 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    plt.title(location)
    plt.legend()
    
plot_scatter_chart(df7,'Yelahanka')

#### Properties should be removed where the price of (for example) 3 bedroom apartment is less than 2 bedroom apartment (with same square ft area) for the same location.

#### 2 BHK apartments whose price_per_sqft is less than mean price_per_sqft of 1 BHK apartment can be removed.

In [ ]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

df8 = remove_bhk_outliers(df7)

df8.shape

#### Plot same scatter chart again to visualize price_per_sqft for 2 BHK and 3 BHK properties.

In [ ]:
plot_scatter_chart(df8,"Yelahanka")

In [ ]:
import matplotlib
matplotlib.rcParams["figure.figsize"] = (10,5)
plt.hist(df8.price_per_sqft,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")


<span style="color:green; font-size:25px;">Outlier Removal: Using Bathrooms Feature</span>

#### It is unusual to have "the number of bathrooms" greater than "the number of bedrooms + 2"

In [ ]:
df8.bath.unique()

In [ ]:
plt.hist(df8.bath,rwidth=0.8)
plt.xlabel("Number of bathrooms")
plt.ylabel("Count")

In [ ]:
df8[df8.bath>10]

In [ ]:
df8[df8.bath > df8.bhk + 2]

In [ ]:
df9 = df8[df8.bath<df8.bhk+2]
df9.shape

In [ ]:
df10 = df9.drop(['size','price_per_sqft'],axis='columns')
df10.head()

<span style="color:green; font-size:25px;">One Hot Encoding for Locations</span>

In [ ]:
one_hot_location = pd.get_dummies(df10.location).astype(int)
one_hot_location.head(2)

In [ ]:
df11 = pd.concat([df10, one_hot_location], axis='columns')
df11.head(2)

In [ ]:
df12 = df11.drop('location', axis='columns')
df12.head(2)

In [ ]:
# df13 = df12.copy()

# columns_to_normalize = ['total_sqft', 'bath', 'price', 'bhk']
# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(df13[columns_to_normalize])
# df13[columns_to_normalize] = scaled_data

<span style="color:green; font-size:25px;">Building the model</span>

In [ ]:
df12.shape

In [ ]:
X = df12.drop(['price'],axis = 'columns')
X.head(2)

In [ ]:
y = df12.price
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 62)

<span style="color:green; font-size:25px;">LinearRegression model</span>

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

<span style="color:green; font-size:25px;">Used K Fold cross validation to measure accuracy of the LinearRegression model</span>

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cross_val = ShuffleSplit(n_splits = 4, test_size = 0.2, random_state = 0)

cross_val_score(LinearRegression(), X, y, cv = cross_val)

<span style="color:green; font-size:25px;">Finding best model using GridSearchCV</span>

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

In [ ]:
def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'fit_intercept': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1, 2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['squared_error', 'friedman_mse'],
                'splitter': ['best', 'random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits = 5, test_size = 0.2, random_state = 0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv = cv, return_train_score = False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model', 'best_score', 'best_params'])

find_best_model_using_gridsearchcv(X,y)

<span style="color:green; font-size:25px;">Testing the model for a few properties</span>

In [ ]:
def predict_price(location,sqft,bath,bhk):    
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    # Created a DataFrame with the same feature names as the training data
    input_data = pd.DataFrame([x], columns=X.columns)
    
    price = lr_clf.predict(input_data)[0] * 10
    return f"{price:.2f} million"

In [ ]:
predict_price('Whitefield',1000, 2, 2)

In [ ]:
predict_price('Whitefield',1000, 3, 3)

In [ ]:
predict_price('Hebbal',1000, 2, 2)

In [ ]:
predict_price('Hebbal',1000, 3, 3)

#### Export the tested model to a pickle file

In [ ]:
import pickle
with open('house_price_model.pickle','wb') as f:
    pickle.dump(lr_clf, f)


#### Export location and column information to a file that will be useful later on in our prediction application

In [ ]:
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("columns.json", "w") as f:
    f.write(json.dumps(columns))